In [1]:
using LinearAlgebra

using DifferentialEquations
using Flux
using DiffEqFlux
using Plots

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1186
┌ Info: Precompiling DiffEqFlux [aae7a2af-3d4f-5e19-a356-7da93b79d9d0]
└ @ Base loading.jl:1186
┌ Info: Recompiling stale cache file /home/gridsan/aramadhan/.julia/compiled/v1.1/DifferentialEquations/UQdwS.ji for DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1184
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1186


In [2]:
const N = 16
const L = 1
const Δx = L / N
const κ = 1

 d = -2 * ones(N)
sd = ones(N-1)
A = Array(Tridiagonal(sd, d, sd))
A[1, N] = 1
A[N, 1] = 1
A_diffusion = (κ/Δx^2) .* A

function diffusion(∂u∂t, u, p, t)
    ∂u∂t .= A_diffusion * u
    return 
end

diffusion (generic function with 1 method)

In [3]:
x = range(-L/2, L/2, length=N)
u₀ = @. exp(-100*x^2)
tspan = (0.0, 0.1)
params = [Δx, κ]

2-element Array{Float64,1}:
 0.0625
 1.0   

In [5]:
datasize = 30
t = range(tspan[1], tspan[2], length=datasize)

0.0:0.0034482758620689655:0.1

In [7]:
prob = ODEProblem(diffusion, u₀, tspan)
ode_data = Array(solve(prob, Tsit5(), saveat=t))

16×30 Array{Float64,2}:
 1.38879e-11  0.000160344  0.00215047  …  0.158223  0.159223  0.160099
 6.99705e-9   0.00087666   0.00636828     0.159464  0.160304  0.161033
 1.44928e-6   0.00462848   0.0199546      0.161636  0.16221   0.16272 
 0.00012341   0.020396     0.0546109      0.164632  0.16482   0.164975
 0.00432024   0.0726686    0.126828       0.167703  0.167515  0.16736 
 0.0621765    0.201762     0.245097    …  0.170699  0.170125  0.169615
 0.367879     0.417262     0.386382       0.172871  0.172031  0.171302
 0.894839     0.611587     0.48795        0.174112  0.173112  0.172236
 0.894839     0.611587     0.48795        0.174112  0.173112  0.172236
 0.367879     0.417262     0.386382       0.172871  0.172031  0.171302
 0.0621765    0.201762     0.245097    …  0.170699  0.170125  0.169615
 0.00432024   0.0726686    0.126828       0.167703  0.167515  0.16736 
 0.00012341   0.020396     0.0546109      0.164632  0.16482   0.164975
 1.44928e-6   0.00462848   0.0199546      0.161636  0

In [8]:
dudt = Chain(Dense(N, 100, tanh),
             Dense(100, N))

Chain(Dense(16, 100, tanh), Dense(100, 16))

In [9]:
ps = Flux.params(dudt)
n_ode = x -> neural_ode(dudt, x, tspan, Tsit5(), saveat=t, reltol=1e-7, abstol=1e-9)

#3 (generic function with 1 method)

In [10]:
pred = n_ode(u₀)

function predict_n_ode()
  n_ode(u₀)
end

predict_n_ode (generic function with 1 method)

In [11]:
loss_n_ode() = sum(abs2, ode_data .- predict_n_ode())

loss_n_ode (generic function with 1 method)

In [12]:
data = Iterators.repeated((), 1000)
opt = ADAM(0.1)

ADAM(0.1, (0.9, 0.999), IdDict{Any,Any}())

In [22]:
cb = function ()  # callback function to observe training
  loss = loss_n_ode()
  display(loss)

  # plot current prediction against data
  # cur_pred = Flux.data(predict_n_ode())

  loss < 0.1 && Flux.stop()
end

cb()

0.9878774926495675 (tracked)

false

In [23]:
Flux.train!(loss_n_ode, ps, data, opt, cb = cb)

1.1244774236232902 (tracked)

1.1655380554515398 (tracked)

0.9650170370991754 (tracked)

0.7130912424715691 (tracked)

0.6420090692789944 (tracked)

...

0.10600311402173368 (tracked)

0.10447453408705486 (tracked)

0.10296188216401464 (tracked)

0.10146584664629645 (tracked)

0.09998677409210671 (tracked)

In [24]:
nn_pred = Flux.data(predict_n_ode())
@gif for n=1:datasize
    plot(x, ode_data[:, n], ylim=(0, 1), label="data", show=false)
    plot!(x, nn_pred[:, n], ylim=(0, 1), label="Neural ODE", show=false)
end

┌ Info: Saved animation to 
│   fn = /home/gridsan/aramadhan/6S898-climate-parameterization/notebooks/tmp.gif
└ @ Plots /home/gridsan/aramadhan/.julia/packages/Plots/Iuc9S/src/animation.jl:95


Plots.AnimatedGif("/home/gridsan/aramadhan/6S898-climate-parameterization/notebooks/tmp.gif")

In [25]:
u1 = @. exp(-10*abs(x))

16-element Array{Float64,1}:
 0.006737946999085467
 0.013123728736940956
 0.025561533206507402
 0.049787068367863944
 0.09697196786440505 
 0.18887560283756188 
 0.36787944117144233 
 0.7165313105737893  
 0.7165313105737893  
 0.36787944117144233 
 0.18887560283756188 
 0.09697196786440505 
 0.049787068367863944
 0.025561533206507402
 0.013123728736940956
 0.006737946999085467

In [37]:
u₀_cos = @. 1 + cos(2π * x)

prob = ODEProblem(diffusion, u₀_cos, tspan)
ode_data = Array(solve(prob, Tsit5(), saveat=t))

nn_pred = Flux.data(n_ode(u₀_cos))
@gif for n=1:datasize
    plot(x, ode_data[:, n], ylim=(0, 2), label="data", show=false)
    plot!(x, nn_pred[:, n], ylim=(0, 2), label="Neural ODE", show=false)
end

┌ Info: Saved animation to 
│   fn = /home/gridsan/aramadhan/6S898-climate-parameterization/notebooks/tmp.gif
└ @ Plots /home/gridsan/aramadhan/.julia/packages/Plots/Iuc9S/src/animation.jl:95


Plots.AnimatedGif("/home/gridsan/aramadhan/6S898-climate-parameterization/notebooks/tmp.gif")

In [52]:
dudt.layers[1].W

Tracked 100×16 Array{Float32,2}:
  0.219476     0.387633   -0.115517    …   0.633287    0.142618    0.479156 
  0.187322     0.346181    0.0562944       0.779724    0.104201    0.612242 
  0.0395474    0.29002     0.146384        0.51009     0.287297    0.307355 
 -0.499803    -0.312583   -0.212347       -0.284019   -0.245401   -0.225501 
  0.117012     0.386874   -0.161308        0.343742    0.251343    0.787597 
 -0.460909    -0.576501   -0.523187    …  -0.22026    -0.582976    0.0193454
 -0.00468038  -0.0315812   0.207367        0.0825084   0.326486   -0.174053 
  0.386303     0.437262    0.613244        0.385165    0.47834     0.102952 
 -0.382644    -0.0915628  -0.37098        -0.232857   -0.384615    0.0556619
 -0.181194     0.167625   -0.0316671       0.117747   -0.162832   -0.164185 
 -0.178336    -0.208751   -0.337404    …  -0.20487    -0.117129   -0.233638 
 -0.544685    -0.104979   -0.00568667     -0.537699   -0.291795   -0.741161 
 -0.106091     0.0180631   0.384981       -